In [1]:
import csv
import pandas as pd

### Lectura de los datos

In [ ]:
'''
Archivos descargados de One Drive: 
V4:MAESTRA_4.0_v09_20230906.xlsx -> hoja MAESTRA_4
V7:20250131_SAIGC_MAESTRA_JE_BENEFICIARIOS_V7 -> hoja 2025-01
V8:20250331_SAIGC_MAESTRA_JE_BENEFICIARIOS_V8 -> hoja 2025-03
'''

sample_data_v4 = pd.read_csv(
    "../Sample_Data/raw/sample_data_4.csv",
    encoding='latin1',
    sep=";",
    on_bad_lines='skip',
    low_memory=False
).head()

sample_data_v4.to_csv("../Sample_Data/raw/sample_data_v4.csv")


sample_data_v7 = pd.read_csv(
    "../Sample_Data/raw/sample_data_7.csv",
    encoding='latin1',
    sep=";",
    on_bad_lines='skip',
    low_memory=False
).head()

sample_data_v7.to_csv("../Sample_Data/raw/sample_data_v7.csv")


sample_data_v8 = pd.read_csv(
    "../Sample_Data/raw/sample_data_8.csv",
    encoding='latin1',
    sep=";",
    on_bad_lines='skip',
    low_memory=False
).head()

sample_data_v8.to_csv("../Sample_Data/raw/sample_data_v8.csv")


### Revision de columnas entre dataframes

***Determinar cuales columnas se comparten entre dataframes (coincidencia exacta de los nombres)*** 

In [ ]:
# Crear sets de columnas
cols_v4 = set(sample_data_v4.columns)
cols_v7 = set(sample_data_v7.columns)
cols_v8 = set(sample_data_v8.columns)

# Todas las columnas únicas en todos los dataframes
all_columns = sorted(cols_v4 | cols_v7 | cols_v8)

# Crear DataFrame de presencia de columnas
column_comparison = pd.DataFrame({
    'column_name': all_columns,
    'in_v4': ['✅' if col in cols_v4 else '❌' for col in all_columns],
    'in_7': ['✅' if col in cols_v7 else '❌' for col in all_columns],
    'in_v8': ['✅' if col in cols_v8 else '❌' for col in all_columns]
})

# Mostrar las primeras filas
column_comparison

,column_name,in_v4,in_7,in_v8
0,ANIO_GRADO_MEDIA,✅,✅,❌
1,ANIO_ICFES,✅,❌,❌
2,ANNO_INF_GRADO_MEDIA,❌,❌,✅
3,BENEFICIARIOS,✅,❌,❌
4,CERTIFICADO_DIPLOMADO,✅,❌,❌
...,...,...,...,...
199,longitude_IDECA,✅,✅,✅
200,nomupl_IDECA,❌,✅,✅
201,nomupz_IDECA,✅,✅,✅
202,ÁREA_DE_CONOCIMIENTO,✅,✅,❌


In [ ]:
# Exportar como .txt con formato tabular
with open("../ouput/column_comparison.txt", "w", encoding="utf-8") as f:
    f.write(column_comparison.to_string(index=False))

***Determinar cuales columnas se comparten entre dataframes a un nivel de significancia dado***

In [ ]:
from rapidfuzz import fuzz, process
import pandas as pd

# Obtener columnas como listas
cols_v4 = list(sample_data_v4.columns)
cols_v7 = list(sample_data_v7.columns)
cols_v8 = list(sample_data_v8.columns)


# Función para buscar columnas similares
# Esta función compara cada columna de un DataFrame con las columnas de otro usando fuzzy matching.
# Retorna las parejas de columnas más similares cuyo puntaje de similitud supera un umbral (por defecto 85).
# Se excluyen coincidencias exactas y se incluye el nombre de cada columna junto con su similarity_score.
def find_similar_columns(source_cols, target_cols, source_name, target_name, threshold=85):
    similar_pairs = []
    for src in source_cols:
        match, score, _ = process.extractOne(src, target_cols, scorer=fuzz.token_sort_ratio)
        if score >= threshold and src != match:  # evitamos los que son exactamente iguales
            similar_pairs.append({
                f'{source_name}_col': src,
                f'{target_name}_col': match,
                'similarity_score': score
            })
    return similar_pairs

#comparacion entre pares de dataframes
results_v4_7 = find_similar_columns(cols_v4, cols_v7, 'v4', 'v7')
results_v4_8 = find_similar_columns(cols_v4, cols_v8, 'v4', 'v8')
result_v7_v8 = find_similar_columns(cols_v7, cols_v8, 'v7', 'v8')

#guardar resultados en dataframes
df_v4_7 = pd.DataFrame(results_v4_7)
df_v4_8 = pd.DataFrame(results_v4_8)
df_v7_v8 = pd.DataFrame(result_v7_v8)

df_v4_7.to_string(open("../ouput/coincidencias_v4_v7.txt", "w", encoding="utf-8"), index=False)
df_v4_8.to_string(open("../ouput/coincidencias_v4_v8.txt", "w", encoding="utf-8"), index=False)
df_v7_v8.to_string(open("../ouput/coincidencias_v7_v8.txt", "w", encoding="utf-8"), index=False)

***Determinar cuales columnas se comparten entre dataframes (coincidencia fuzzy wuzzy)***

In [ ]:
from rapidfuzz import fuzz, process
import pandas as pd

# Obtener columnas como listas
cols_v4 = list(sample_data_v4.columns)
cols_v7 = list(sample_data_v7.columns)
cols_v8 = list(sample_data_v8.columns)

# Función para buscar columnas similares entre DataFrames
def compare_all_columns(source_cols, target_cols, source_name, target_name, threshold=85):
    all_pairs = []
    for src in source_cols:
        for tgt in target_cols:
            score = fuzz.token_sort_ratio(src, tgt)
            if score >= threshold:  # Consideramos solo pares con score >= threshold
                all_pairs.append({
                    f'{source_name}_col': src,
                    f'{target_name}_col': tgt,
                    'similarity_score': score
                })
    return all_pairs

# Comparar entre pares de DataFrames
results_v4_v7 = compare_all_columns(cols_v4, cols_v7, 'v4', 'v7')
results_v4_v8 = compare_all_columns(cols_v4, cols_v8, 'v4', 'v8')
results_v7_v8 = compare_all_columns(cols_v7, cols_v8, 'v7', 'v8')

# Crear DataFrames de resultados
df_v4_v7 = pd.DataFrame(results_v4_v7)
df_v4_v8 = pd.DataFrame(results_v4_v8)
df_v7_v8 = pd.DataFrame(results_v7_v8)

# Crear una lista de todas las columnas únicas de los 3 DataFrames
all_columns = sorted(set(cols_v4 + cols_v7 + cols_v8))

# Obtener conjuntos de columnas que aparecen en pares fuzzy-similares
sim_v4_v7_cols = set(df_v4_v7['v4_col']).union(set(df_v4_v7['v7_col']))
sim_v4_v8_cols = set(df_v4_v8['v4_col']).union(set(df_v4_v8['v8_col']))
sim_v7_v8_cols = set(df_v7_v8['v7_col']).union(set(df_v7_v8['v8_col']))

# Crear tabla con checks de similitud
similarity_table = pd.DataFrame({
    'column_name': all_columns,
    'v4_v7_similar': ['✅' if col in sim_v4_v7_cols else '❌' for col in all_columns],
    'v4_v8_similar': ['✅' if col in sim_v4_v8_cols else '❌' for col in all_columns],
    'v7_v8_similar': ['✅' if col in sim_v7_v8_cols else '❌' for col in all_columns]
})

# Mostrar resultado
similarity_table.head(20)  # podés cambiar el número o quitarlo para ver todo

# Exportar como .txt con formato tabular
with open("../ouput/similarity_table.txt", "w", encoding="utf-8") as f:
    f.write(similarity_table.to_string(index=False))


### Listado de columnas que a priori considero que vulnerables

In [ ]:
#Listar la columnas
#sample_data.columns.tolist()

for col in sample_data.columns:
    print(f"{col}: {sample_data[col].iloc[0]}")

'''

COlUMNAS QUE A PRIORI, A MI CONSIDERACION REPRESENTAN UNA VULNERABILIDAD A LA PRIVACIDAD

id_persona
ju_jovenes_id
oferta_id
codigo_snies_ies
nombre_institucion_superior
codigo_snies_programa
nombre_programa
nivel_de_formacion
departamento_residencia_sicore
municipio_residencia_sicore
zona_sicore
codloc_sicore #?
localidad_sicore
codloc_inscritos #?
localidad_inscritos
codupl_ideca #?
nomupl_ideca #?
codupz_ideca #codigo upz
nomupz_ideca #nombre upz
direccion_residencia ***
edad_del_calculo #?
fecha_nacimiento *** 
sexo ***
sexo_sicore
identidad_genero
orientacion_sexual ***
etnia ***
grupo ***
estrato ***
tipo_documento_icfes ***
numero_documento_icfes ***
tipo_documento ***
numero_documento ***
primer_nombre ***
segundo_nombre ***
primer_apellido ***
segundo_apellido ***
telefono_principal ***
correo_electronico ***
discapacidad_sicore
tipo_discapacidad_sicore
estado_civil ***
hijos ***
responsable_hijos
ra_victima_conflicto #?
hecho_victimizante #?
relacion_victima #?
ra_violencia_mujer 
puntaje_sisben ****
sisben4_grupo
sisben4_nivel
sisben4_clasificacion
ra_discapacidad #?
tipo_discapacidad_saludcapital 
minsalud_condiciondiscapacidad ***
minsalud_cat_fisica ***
minsalud_cat_visual ***
minsalud_cat_auditiva ***
minsalud_cat_intelectual *** 
minsalud_cat_psicosocial ***
minsalud_cat_sordoceguera ***
minsalud_cat_multiple ***
ra_cosejero_juventud_electo #?
ra_reto_u_finalizado #?
ra_inmersion_media #?
ra_reto_sdis #?
ra_parceros #?
ra_certificado_cisco #?
ra_deportista_idrd #?
deporte #?
municipio_grado_media #?
anno_inf_grado_media #?
codigo_dane_grado_media #?
nombre_establecimiento_grado_media ***
codigo_dane_sede_grado_media #?
nombre_sede_grado_media ***
secor_grado_media #?
origen_grado_media #?
saber11_periodo 
saber11_puntaje_global
saber11_puesto
saber11_percentil_global
n_reg_corte #?
ultimo_estado_corte #?
ultimo_estado_ajustado_corte #?
e_beneficiario_corte #?
convocatoria_corte #?
tipo_linea_corte #?
peama_corte #?
codigo_institucion_corte #?
nombre_institucion_corte #?
sector_corte #?
codigo_snies_programa_corte ***
nombre_del_programa_corte ***
nucleo_basico_del_conocimiento_corte
cine_f_2013_ac_campo_amplio_corte
cine_f_2013_ac_campo_especifico_corte
cine_f_2013_ac_campo_detallado_corte
codloc_corte
'''

ID_MAESTRA: 1
HV_SICORE_CORTE: NO_APLICA
CONVOCATORIA: JU1
ESTADO_HOJA_VIDA: DILIGENCIAMIENTO
ID_PERSONA: 247416.0
JU_JOVENES_ID: 3288.0
OFERTA_ID: 184.0
LINEA_ACCESO: ACCESO
TIPO_ADMISION: ADMISION ESPECIAL AMPLIADA
INTERVENCION: CONVENCIONAL
ESTRATEGIA: FLEXIBLE
CODIGO_SNIES_IES: 1714
NOMBRE_INSTITUCION_SUPERIOR: COLEGIO MAYOR DE NUESTRA SEÑORA DEL ROSARIO
SECTOR: Privado
CODIGO_SNIES_PROGRAMA: 4368
NOMBRE_PROGRAMA: CIENCIA POLITICA Y GOBIERNO
NIVEL_DE_FORMACIÓN: Universitario
CODIGO_SNIES_HOMOLOGACION: nan
PROGRAMA_HOMOLOGACION: nan
MODALIDAD: Presencial
DEPARTAMENTO_RESIDENCIA_SICORE: BOGOTÁ D.C.
MUNICIPIO_RESIDENCIA_SICORE: BOGOTA D.C.
ZONA_SICORE: URBANA
codloc_SICORE: 19.0
LOCALIDAD_SICORE: CIUDADBOLIVAR
codloc_INSCRITOS: 19.0
LOCALIDAD_INSCRITOS: CIUDADBOLIVAR
codupl_IDECA: UPL03
nomupl_IDECA: Arborizadora
codupz_IDECA: UPZ70
nomupz_IDECA: JERUSALEM
longitude_IDECA: -74,15973786
latitude_IDECA: 4,566587332
DIRECCION_RESIDENCIA: TRANSVERSAL 40 # 70 A - 12 Sur
EDAD_DEL_CALCULO: 1